In [119]:
import os.path

import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
import warnings
warnings.filterwarnings("ignore")

In [120]:
mnist_dataset,mnist_info = tfds.load(name='mnist',with_info=True,as_supervised=True)

mnist_train,mnist_test = mnist_dataset['train'],mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples,tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples,tf.int64)

def scale(image, label):
    image = tf.cast(image,tf.float32)
    image /=255.
    return image, label

scaled_train_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

Buffer_Size = 4500

shuffled_train_validation_data = scaled_train_validation_data.shuffle(Buffer_Size)

validation_data = shuffled_train_validation_data.take(num_validation_samples)
train_data = shuffled_train_validation_data.skip(num_validation_samples)

Batch_Size = 150
train_data = train_data.batch(Batch_Size)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs,validation_targets = next(iter(validation_data))


2022-01-21 22:03:38.811082: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [121]:
input_size = 784
output_size = 10
hidden_layer_size = 250

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='tanh'),
                            tf.keras.layers.Dense(output_size,activation='softmax')
                            ])

In [122]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [123]:
num_epochs=25
model.fit(train_data,epochs=num_epochs,validation_data=(validation_inputs,validation_targets),verbose=2)

Epoch 1/25
360/360 - 7s - loss: 0.2846 - accuracy: 0.9131 - val_loss: 0.1707 - val_accuracy: 0.9498 - 7s/epoch - 20ms/step
Epoch 2/25
360/360 - 5s - loss: 0.1360 - accuracy: 0.9585 - val_loss: 0.1233 - val_accuracy: 0.9608 - 5s/epoch - 13ms/step
Epoch 3/25
360/360 - 4s - loss: 0.0978 - accuracy: 0.9698 - val_loss: 0.0906 - val_accuracy: 0.9753 - 4s/epoch - 11ms/step
Epoch 4/25
360/360 - 4s - loss: 0.0693 - accuracy: 0.9781 - val_loss: 0.0892 - val_accuracy: 0.9727 - 4s/epoch - 11ms/step
Epoch 5/25
360/360 - 5s - loss: 0.0582 - accuracy: 0.9820 - val_loss: 0.0764 - val_accuracy: 0.9765 - 5s/epoch - 13ms/step
Epoch 6/25
360/360 - 5s - loss: 0.0408 - accuracy: 0.9868 - val_loss: 0.0613 - val_accuracy: 0.9802 - 5s/epoch - 13ms/step
Epoch 7/25
360/360 - 6s - loss: 0.0351 - accuracy: 0.9889 - val_loss: 0.0713 - val_accuracy: 0.9782 - 6s/epoch - 15ms/step
Epoch 8/25
360/360 - 5s - loss: 0.0298 - accuracy: 0.9900 - val_loss: 0.0572 - val_accuracy: 0.9843 - 5s/epoch - 13ms/step
Epoch 9/25
360/3

In [124]:
model.save("handwritten.model")

2022-01-21 22:05:42.921240: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


INFO:tensorflow:Assets written to: handwritten.model/assets


INFO:tensorflow:Assets written to: handwritten.model/assets


In [143]:
import cv2
import numpy as np
image_number=9
model_my = tf.keras.models.load_model('handwritten.model')

try:
        img = cv2.imread('digits/digit{}.png'.format(image_number))[:,:,0]
        img = np.invert(np.array([img]))
        predictions = model.predict(img)
        print(f"This digit is probably a {np.argmax(predictions)}")
        plt.imshow(img[0],cmap=plt.cm.binary)
        plt.show()
except:
        print("Error")
finally:
        image_number+=1


This digit is probably a 7
Error
